# PROJET PYTHON POUR LA DATA SCIENCE 2022

# THEME : ANALYSE DES AVIS DES STAGIAIRES ET EMPLOYES DE SOCIETE GENERALE TRANSMIS VIA LA PLATEFORME INDEED.COM ENTRE 2012 ET 2022  

## Auteurs:
* TALEB AHMED Raja
* MAGAJIE WAMSA Berthe Magella
* RENE Léo

## Scrapping des commentaires des employés et stagiaires de Société Générale sur le site Indeed.com

Le but de cette partie est de récupérer par webscrapping les données devant servir à notre analyse. Il s'agit notamment des retours d'expérience et commentaires des employés et stagiaires de SOCIETE GENERALE transmis sur la plateforme Indeed.com entre 2012 et 2022.

Pour récupérer nos données sur le site Indeed, nous utilisons le package Selenium, étant donné que le site web n'est pas statique. Le pilote Web utilisé est Chrome.

In [28]:
# Installation des packages utiles
!pip install selenium
!pip install datefinder

In [29]:
#Importation des librairies utiles
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Pour récupérer les commentaires par pays, nous avons ouvert individuellement les pages correspondantes. Trois fichiers .txt ont été créés pour contenir les données récupérées par scrapping pour chaque pays : **SG-FR.txt** pour la France, **SG-US.txt** pour les US et **SG-IN.txt** pour l'Inde. 

In [37]:
# Ouverture du site Indeed et des pages correspondant aux trois pays en utilisant chromedriver
sites=['https://fr.indeed.com/cmp/Soci%C3%A9t%C3%A9-G%C3%A9n%C3%A9rale-1/reviews?fcountry=US' , 'https://fr.indeed.com/cmp/Soci%C3%A9t%C3%A9-G%C3%A9n%C3%A9rale-1/reviews?fcountry=IN']
fichierFR = open("SG-FR.txt", "w", encoding="utf-8")
fichierUS = open("SG-US.txt", "w", encoding="utf-8")
fichierIN = open("SG-IN.txt", "w", encoding="utf-8")
fichiers = [fichierFR,fichierUS, fichierIN]
pays = ['France', 'US', 'Inde']

WebDriverException: Message: Service chromedriver unexpectedly exited. Status code was: 127


**IMPORTANT:** Nous avons chargé le chromedriver (pilote) sur Github et il est disponible à l'url suivante: https://github.com/BertheMagella/Projet_Python_pour_la_Data_Science/blob/main/chromedriver.exe

Cependant, ce pilote ne peut s'exécuter que s'il est dans le même dossier que l'exécutable du notebook. Il est à cet effet recommandé de télécharger ce pilote, de le mettre dans le même dossier que l'exécutable de ce notebook et enfin de mettre à jour le chemin d'accès du pilote (driver_path).

In [29]:
# Ouverture du site Indeed en utilisant Chromedriver et scrapping
driver=webdriver.Chrome('chromedriver')

for j in range(len(sites)):
    driver.get(sites[j])
    if j == 0 :
        # autoriser cookies
        cookies_xpath = '//*[@id="onetrust-accept-btn-handler"]'
        driver.find_element(By.XPATH, cookies_xpath).click()
    
    fichier = fichiers[j]
    from selenium.webdriver.remote.errorhandler import NoSuchElementException
    i = 1
    while (True):
        xpath = '//*[@id="cmp-container"]/div/div[1]/main/div[2]/div[1]/div/div[' + str(i) + ']'
        try :
            metier = driver.find_element(By.XPATH, xpath)
            fichier.write(metier.text+ 2*'\n')
        
            sep = 10*'#'
            fichier.write(sep + 2*'\n')
            i=i+1
        
        except NoSuchElementException :
            
            buttons = driver.find_elements(By.XPATH, '//*[@id="cmp-container"]/div/div[1]/main/div[2]/div[1]/div/nav/ul/li')
            if buttons[-1].text == 'Suivant':
                buttons[-1].click()
               
            else :
                break
            time.sleep(20)
            i=1
            
    fichier.close()
    

NameError: name 'driver' is not defined

# Création et mise en forme de la base de données 

Dans cette partie, un dataframe est constitué à partir des données scrappées et stockées au préalable dans les fichiers **SG-FR.txt**, **SG-US.txt** et **SG-IN.txt** ; la base de données ainsi construite comporte 2506 lignes/observations pour 8 variables que sont  **'country'** qui renferme le pays concerné (France, US ou Inde), **'rating'** qui renferme les notes attribuées (sur 5), **'contract'** pour le type de contrat(Stage, cdd, cdi),  **'is_employed'** qui prend la valeur "True" si l'individu travaille au sein de la SG au moment du commentaire et "False" sinon, **'job'** pour le poste occupé, **'city'** pour le lieu de travail (la ville) , **'date'** pour la date de publication du commentaire et **'comment'** pour les commentaires.

In [88]:
def find_date(text):
    months = ['janvier ', 'février ', 'mars ', 'avril ', 'mai ', 'juin ', 'juillet ', 'août ', 'septembre ', 
              'octobre ', 'novembre ', 'décembre ']

    for month in months:
        index = text.rfind(month)
        if index == -1:
            continue
            
        else :
            year =  text[index + len(month) :]
            year = year.strip(' ')
            assert year.isdigit() and len(year)==4 , f"{year} doit être une année"
            return int(year)
            
            
    return None


def find_contract(text):
    text = text.casefold()
    for contract in ['cdi', 'cdd']:
        if contract in text:      
            return contract
        
    for contract in ['stage', 'stagiaire', 'internship', 'intern']:
        if contract in text:      
            return 'stage'
        
    return None

In [89]:
df = pd.DataFrame(columns= ['Country', 'rating', 'contract', 'is_employed', 'job', 'city', 'date', 'comment'])
pays = ['France', 'US', 'Indie']
paths = ["SG-FR.txt","SG-US.txt", "SG-IN.txt"]
for j in range(len(paths)):
    fichier = open(paths[j], 'r', encoding="utf-8")
    
    text = fichier.read()
    parts = text.split('\n##########\n\n')
    errors = {
        'rating': [],
        'job_city_date': []        
    }
    
    for avis in parts:
        new_row = {}
        avis_par_ligne = avis.split('\n')
        i = 0
        
        # recherche du rating
        
        rating = avis_par_ligne[i]
        # convertir par example '3,0' à 3.0
        rating = rating.replace(',', '.')
        
        try:
            # si la première ligne correspond à un classement, on l'enregistre
            rating = float(rating)

            # passage à la ligne suivante
            i+=1
            
        except ValueError:
            errors['rating'].append(avis)
            continue
            

        # recherche de la ligne contenant le poste la ville et la date de l'avis 
        # en cherchant chaque fois le contrat de l'employé
        contract = None
        no_date = True
        # tant qu'une date n'a pas été détectée on passe à la ligne suivante
        while(no_date and i<len(avis_par_ligne)):
            
            job_city_date = avis_par_ligne[i]
            year = find_date(job_city_date)
            # si aucune date est détectée find_date renvoie None
            no_date = year is None
              
            contract = contract or find_contract(job_city_date)
            i+=1
            
        # si aucune date n'a été trouvé. Il ne s'agit peut être pas d'un avis
        if no_date:
            errors['job_city_date'].append(avis)
            continue
        else :
            
            mark1 = '(employé actuel)'
            mark2 = '(ancien employé)'

            is_employed = mark1 in job_city_date

            mark = mark1 if is_employed else mark2
            index = job_city_date.find(mark)
            
            job = job_city_date[:index].strip(' ')
            
            city = '-'.join(job_city_date[index:].split('-')[1:-1]).strip(' ')
            
        comment = ''
        while avis_par_ligne[i] in ['Avis à la une', "L'avis le plus utile, sélectionné par Indeed"]:
            i+=1
            
            
        while i<len(avis_par_ligne) and  avis_par_ligne[i]!= 'Avez-vous trouvé cet avis utile ?':
        
            comment += avis_par_ligne[i] + '\n'
            i+=1
        
            
        new_row = {
            'Country': pays[j],
            'rating': rating,
            'contract': contract,
            'is_employed': is_employed,
            'job': job,
            'city': city,
            'date': year,    
            'comment': comment
        } 
        df = df.append(new_row, ignore_index=True)
    fichier.close()

/tmp/ipykernel_180/1948063583.py:91: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_180/1948063583.py:91: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_180/1948063583.py:91: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_180/1948063583.py:91: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
/tmp/ipykernel_180/1948063583.py:91: FutureWarning: In a future version, object-dtype columns with a

In [90]:
#suppression de la redondance
df.drop_duplicates(inplace=True, ignore_index=True)

In [91]:
df

,Country,rating,contract,is_employed,job,city,date,comment
0,France,3.0,cdd,False,Conseillère clientèle,Massy (91),2022,"Bonne équipe, dommage que la direction soit bl..."
1,France,5.0,None,False,Responsable,Falaise,2022,"Équipe au top, l'esprit d'équipe fait la force..."
2,France,4.0,None,False,Compliance Officer,Nanterre (92),2022,"bonne ambiance, bonne équipe, bon relationnel,..."
3,France,5.0,None,False,Assistante administrative,Saint-Dié-des-Vosges (88),2022,"Missions variées, solidarité dans l’équipe et ..."
4,France,4.0,None,False,Chargée de communication,La Défense (92),2022,Première expérience professionnelle durant 3 a...
...,...,...,...,...,...,...,...,...
2515,Indie,4.0,None,True,Business Analyst,"Bangalore, KA",2012,A good working environment with good work-life...
2516,Indie,5.0,None,True,Auditor,Mumbai,2012,"SA 8000, EMS & Social Audits\nBPP Audits in Di..."
2517,Indie,5.0,None,False,Software Engineer,"Bangalore, KA",2012,nice banking bussiness. had a great time worki...
2518,Indie,5.0,None,False,Software Engineer,"Bangalore, KA",2012,This is a euro bank and i had great fun in lea...


In [92]:
#Nombre de modalités pour chacune des variables
df.nunique()

Country           3
rating            5
contract          3
is_employed       2
job            1706
city            803
date             11
comment        2518
dtype: int64

In [93]:
#Vérification valeurs manquantes
df.count()

Country        2520
rating         2520
contract        237
is_employed    2520
job            2520
city           2520
date           2520
comment        2520
dtype: int64

In [94]:
#Vérification types des variables
print(df.dtypes)

Country         object
rating         float64
contract        object
is_employed     object
job             object
city            object
date            object
comment         object
dtype: object


In [95]:
#Création d'une copie de la base
df_new = df.copy()

In [96]:
#Suppression des codes postaux dans la colonne city 
characters = ")(1234567890"
df_new['city'] = df['city'].apply(lambda x: x.strip(characters))
df_new

,Country,rating,contract,is_employed,job,city,date,comment
0,France,3.0,cdd,False,Conseillère clientèle,Massy,2022,"Bonne équipe, dommage que la direction soit bl..."
1,France,5.0,None,False,Responsable,Falaise,2022,"Équipe au top, l'esprit d'équipe fait la force..."
2,France,4.0,None,False,Compliance Officer,Nanterre,2022,"bonne ambiance, bonne équipe, bon relationnel,..."
3,France,5.0,None,False,Assistante administrative,Saint-Dié-des-Vosges,2022,"Missions variées, solidarité dans l’équipe et ..."
4,France,4.0,None,False,Chargée de communication,La Défense,2022,Première expérience professionnelle durant 3 a...
...,...,...,...,...,...,...,...,...
2515,Indie,4.0,None,True,Business Analyst,"Bangalore, KA",2012,A good working environment with good work-life...
2516,Indie,5.0,None,True,Auditor,Mumbai,2012,"SA 8000, EMS & Social Audits\nBPP Audits in Di..."
2517,Indie,5.0,None,False,Software Engineer,"Bangalore, KA",2012,nice banking bussiness. had a great time worki...
2518,Indie,5.0,None,False,Software Engineer,"Bangalore, KA",2012,This is a euro bank and i had great fun in lea...


In [97]:
df_new.nunique()

Country           3
rating            5
contract          3
is_employed       2
job            1706
city            795
date             11
comment        2518
dtype: int64

In [98]:
#Ajustement de la police
for i in range(2506):
    df_new['city']=df_new['city'].replace(df_new['city'][i], df_new['city'][i].capitalize())
    df_new['comment']=df_new['comment'].replace(df_new['comment'][i], df_new['comment'][i].capitalize())

In [99]:
df_new.nunique()

Country           3
rating            5
contract          3
is_employed       2
job            1706
city            760
date             11
comment        2518
dtype: int64

In [100]:
df_new.to_csv(r'Database.csv', index=False, header=True)

In [101]:
pd.read_csv(r'Database.csv')

,Country,rating,contract,is_employed,job,city,date,comment
0,France,3.0,cdd,False,Conseillère clientèle,Massy,2022,"Bonne équipe, dommage que la direction soit bl..."
1,France,5.0,NaN,False,Responsable,Falaise,2022,"Équipe au top, l'esprit d'équipe fait la force..."
2,France,4.0,NaN,False,Compliance Officer,Nanterre,2022,"Bonne ambiance, bonne équipe, bon relationnel,..."
3,France,5.0,NaN,False,Assistante administrative,Saint-dié-des-vosges,2022,"Missions variées, solidarité dans l’équipe et ..."
4,France,4.0,NaN,False,Chargée de communication,La défense,2022,Première expérience professionnelle durant 3 a...
...,...,...,...,...,...,...,...,...
2515,Indie,4.0,NaN,True,Business Analyst,"Bangalore, ka",2012,A good working environment with good work-life...
2516,Indie,5.0,NaN,True,Auditor,Mumbai,2012,"SA 8000, EMS & Social Audits\nBPP Audits in Di..."
2517,Indie,5.0,NaN,False,Software Engineer,"Bangalore, ka",2012,nice banking bussiness. had a great time worki...
2518,Indie,5.0,NaN,False,Software Engineer,"Bangalore, ka",2012,This is a euro bank and i had great fun in lea...
